## 데이터 전처리 : 재무제표 데이터 가공
* 재무적 부실이 아닌 다른 이유로 상장이 폐지된 기업들은 분석 대상에서 배제된다. 이러한 기업들은 부실기업이나 정상기업의 범주에 속하지 않으므로, 데이터 분석에 포함시키지 않는다.
* 상장폐지 기업 중 재무건전성 문제가 아닌 다른 사유로 상장이 폐지되는 경우
 - 합병,회사의 해산,주식 유통량 부족,코넥스로의 이전,자발적 상장 철회

#### 전처리 과정
1. **CSV 파일 읽기:** 두 개의 CSV 파일을 'cp949' 인코딩으로 읽어들임.

2. **상장폐지 데이터 추출 및 병합:** 상장폐지일이 있는 데이터를 추출하고, 상장폐지 사유 데이터와 병합.

3. **불필요한 데이터 제거:** '코넥스', '합병' 사유 데이터와 2011년 이전 데이터, 비 12월 회계년도 데이터, 특정 거래소코드를 가진 데이터 제거.

4. **날짜 데이터 변환 및 필터링:** 날짜 관련 열을 datetime 형식으로 변환하고, 필요한 날짜 조건에 맞는 데이터만 유지.

5. **최종 데이터 저장:** 처리된 데이터를 새로운 CSV 파일로 저장.

In [1]:
import pandas as pd
import numpy as np

연결 = pd.read_csv("../data/0. 연결재무제표.csv", encoding='cp949')
개별 = pd.read_csv("../data/1. 코스닥 기업 개별재무제표.csv", encoding='cp949')

# 상장폐지 종목 데이터 추출
df_default =연결[~연결["상장폐지일"].isnull()]

# 2022년도 상장폐지 데이터 보존
indexes_2022_delisting = df_default[df_default["상장폐지일"].str.contains('2022')].index
for i in indexes_2022_delisting:
   df_default.loc[i, "상장폐지일"]= np.NaN
   df_default.loc[i, "소속코드"]=5

In [2]:
# 상장폐시 사유 데이터
df_default_mod = pd.read_csv("../data/1. 상장폐지종목_사유등록.csv",encoding='cp949',index_col=0)
df = pd.concat([연결,df_default_mod["상장폐지사유"]],axis=1)

상장폐지 사유 전처리

In [3]:
# 코넥스 데이터 삭제
drop_index = df[df["상장폐지사유"].str.contains('코넥스') == 1]["상장폐지사유"].index
df.drop(drop_index, inplace=True)

In [4]:
# 피흡수 합병 데이터 삭제
drop_index_2 = df[df["상장폐지사유"].str.contains('합병') == 1]["상장폐지사유"].index
df.drop(drop_index_2, inplace=True)

In [5]:
# 정상데이터 소속코드 수정
mod_index = df[~(df["상장폐지사유"].isnull())]["소속코드"].index

for i in mod_index:
    df.loc[i, "소속코드"]=5

In [6]:
# 상장폐지년도가 2011년 이전 데이터 삭제
# 날짜데이터 datetime화
df["상장일"]=pd.to_datetime(df['상장일'])
df["상장폐지일"]=pd.to_datetime(df['상장폐지일'])
df["회계년도"] = pd.to_datetime(df["회계년도"], format='%Y/%m')

# 상장폐지년도 2011년도 이전 데이터 삭제
df[df["상장폐지일"].dt.year <  2011].index

Index([3464], dtype='int64')

In [7]:
# 회계년도 12월이 아닌 행 삭제
df["회계월"]=df["회계년도"].dt.month
ind =df[~(df["회계월"]==12)].index

In [8]:
# 외국계 기업 삭제
df.drop(ind,inplace=True)
drop = df[df["거래소코드"] > 900000].index
df.drop(drop, inplace=True)
df.reset_index(drop='index',inplace=True)

In [9]:
# 상장 전 데이터 삭제
df["상장년도"]  = df["상장일"].dt.year
df["회계년"]  = df["회계년도"].dt.year
df["연차"] = df["회계년"] - df["상장년도"] 
index_a = df[df["연차"]<=0].index
df.drop(index_a,inplace=True)

In [10]:
# 거래소코드 중복 제거
b = df[["회사명","거래소코드"]]
d = b.groupby("회사명")["거래소코드"].unique()
d = pd.DataFrame(d)

for i in d.index:
    if len(d.loc[i,"거래소코드"])== 2:
        print(i)

# (주)원익피앤이 거래소코드 131390 소프트캠프(주) 210610 원텍(주) 216280 제거
index_b = df[df["거래소코드"]== 131390].index
df.drop(index_b,inplace=True)
index_b = df[df["거래소코드"]== 210610].index
df.drop(index_b,inplace=True)
index_b = df[df["거래소코드"]== 216280 ].index
df.drop(index_b,inplace=True)

df.reset_index(drop='index',inplace=True)

(주)원익피앤이
소프트캠프(주)
원텍(주)


In [11]:
df.to_csv('../data/1. 상장폐지종목 정리끝.csv')

In [12]:
# 아직 뒤에 전처리 해야할 거 많음.
print('데이터 총 기업수 :',len(df["회사명"].unique()))
print('상장폐지 기업수(중복포함):',len(df[~df["상장폐지일"].isnull()]["회사명"]))
print('상장폐지 기업수(중복미포함):',len(df[~df["상장폐지일"].isnull()]["회사명"].unique()))

데이터 총 기업수 : 1451
상장폐지 기업수(중복포함): 860
상장폐지 기업수(중복미포함): 189
